#zipファイルを解凍し、展開する（ディレクトリがローカルになってる）

In [ ]:
!unzip /content/drive/MyDrive/Aidemy/成果物/Hazelnut.zip -d /content/drive/MyDrive/Aidemy/成果物/

unzip:  cannot find or open /content/drive/MyDrive/Aidemy/成果物/Hazelnut.zip, /content/drive/MyDrive/Aidemy/成果物/Hazelnut.zip.zip or /content/drive/MyDrive/Aidemy/成果物/Hazelnut.zip.ZIP.


#転移学習のコード。

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from google.colab import files

# パスの指定
path_train_good = os.listdir('/content/drive/MyDrive/Aidemy/成果物/Hazelnut/train/good')
path_train_crack = os.listdir("/content/drive/MyDrive/Aidemy/成果物/Hazelnut/train/crack")
path_test_good = os.listdir('/content/drive/MyDrive/Aidemy/成果物/Hazelnut/test/good')
path_test_crack = os.listdir("/content/drive/MyDrive/Aidemy/成果物/Hazelnut/test/crack")

#格納場所の作成
img_train_good = []
img_train_crack = []
img_test_good = []
img_test_crack = []

#画像を格納
for i in range(len(path_train_good)):
    img = cv2.imread('/content/drive/MyDrive/Aidemy/成果物/Hazelnut/train/good/' + path_train_good[i])
    b,g,r = cv2.split(img)
    img = cv2.merge([r,g,b])
    img_train_good.append(img)

for i in range(len(path_train_crack)):
    img = cv2.imread('/content/drive/MyDrive/Aidemy/成果物/Hazelnut/train/crack/' + path_train_crack[i])
    b,g,r = cv2.split(img)
    img = cv2.merge([r,g,b])
    img_train_crack.append(img)

for i in range(len(path_test_good)):
    img = cv2.imread('/content/drive/MyDrive/Aidemy/成果物/Hazelnut/test/good/' + path_test_good[i])
    b,g,r = cv2.split(img)
    img = cv2.merge([r,g,b])
    img_test_good.append(img)

for i in range(len(path_train_crack)):
    img = cv2.imread('/content/drive/MyDrive/Aidemy/成果物/Hazelnut/test/crack/' + path_test_crack[i])
    b,g,r = cv2.split(img)
    img = cv2.merge([r,g,b])
    img_test_crack.append(img)


##学習データを水増し

# ImageDataGeneratorクラスのオブジェクト生成
datagen = ImageDataGenerator(
        rotation_range=45, #±45°でランダムに回転
        vertical_flip=True, #垂直方向にランダムで反転
        horizontal_flip=True)#水平方向にランダムで反転

def images_gen(X_list, y_list):
    X_list_add = []
    y_list_add = []
    for X, y in zip(X_list, y_list):
        X = X.reshape((1,) + X.shape)  # 元の画像を4次元に変形

        i = 0
        for batch in datagen.flow(X, batch_size=1):
            batch = batch[0].astype(np.uint8)  # バッチから画像を取り出し、データ型を揃える
            X_list_add.append(batch)
            y_list_add.append(y)
            i += 1
            if i > 19:  # 1枚から20枚生成
                break

    X_add = np.array(X_list_add)
    y_add = np.array(y_list_add)
    return X_add, y_add

X_train = np.array(img_train_crack + img_train_good)
y_train =  np.array([0]*len(img_train_crack) + [1]*len(img_train_good))
X_test = np.array((img_test_crack + img_test_good))
y_test = np.array([0]*len(img_test_crack) + [1]*len(img_test_good))

#画像をリサイズする（元画像は800＊800）リサイズの大きさも工夫してみる価値あり
X_train = np.array([cv2.resize(img, (100, 100)) for img in X_train])
X_test = np.array([cv2.resize(img, (100, 100)) for img in X_test])

#水増し
X_train,y_train = images_gen(X_train,y_train)
X_test,y_test = images_gen(X_test,y_test)

# 正解ラベルをone-hotの形にします(ダミー変数を用いた変換)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# 転移学習のモデルにvgg16を使用
input_tensor = Input(shape=(100, 100, 3))
vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)

# vgg16のoutputを受け取り、2クラス分類する層を定義
top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(2, activation='softmax'))

# vgg16とtop_modelを連結
model = Model(vgg16.inputs, top_model(vgg16.output))

# vgg16の層の重みを変更不能に
for layer in model.layers[:19]:
    layer.trainable = False

# コンパイル
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

# 学習を行います（batch_size,epochsの調整余地あり）
history = model.fit(X_train, y_train, batch_size=50, epochs=5, validation_data=(X_test, y_test))

# ナッツの画像を一枚受け取り、不良品かどうかを判定する関数
# 不良品は0、良品は1となるようにラベルの付与を行なっています。

def hazelnut(img):
    pred = np.argmax(model.predict(np.array([img])))
    if pred == 0:
        return '不良品'
    else:
        return '良品'

# 精度の評価
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

#以下、可視化のためのコード
train_loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
epochs =len(train_loss)

fig = plt.figure(figsize=(15,5))
plt.subplots_adjust(wspace=0.4, hspace=0.6)
ax1 = fig.add_subplot(1,2,1)
ax1.plot(range(epochs), train_loss, marker = '.', label = 'train_loss')
ax1.plot(range(epochs), val_loss, marker = '.', label = 'val_loss')
ax1.legend(loc = 'best')
ax1.set_xlabel('epoch')
ax1.set_ylabel('loss')

ax2 = fig.add_subplot(1,2,2)
ax2.plot(range(epochs), acc, label="acc", ls="-", marker=".")
ax2.plot(range(epochs), val_acc, label="val_acc", ls="-", marker=".")
ax2.set_ylabel("accuracy")
ax2.set_xlabel("epoch")
ax2.legend(loc="best")

plt.show()

# データの可視化（検証データの先頭の10枚）
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(X_test[i].reshape((100,100,3)), 'gray')
plt.suptitle("10 images of test data",fontsize=20)
plt.show()

# 予測（検証データの先頭の10枚）
pred = np.argmax(model.predict(X_test[0:10]), axis=1)
print(pred)

#resultsディレクトリを作成
result_dir = 'results'
if not os.path.exists(result_dir):
    os.mkdir(result_dir)
# 重みを保存
model.save(os.path.join(result_dir, 'model.h5'))

files.download( '/content/results/model.h5' )

58889256/58889256 [==============================] - 0s 0us/step


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 100, 100, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 100, 100, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 50, 50, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 50, 50, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 50, 50, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 25, 25, 128)       0     

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
